# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
from pprint import pprint, pformat
import numpy as np
from toolz import pipe

from leap_ec.individual import Individual
from leap_ec.ops import pool, cyclic_selection, clone

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation, add_segment, remove_segment, copy_segment

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[1, 0, 1, 1, 1], [0, 0, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]
1 [[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
2 [[1, 0, 0, 1, 1], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0]]
3 [[1, 0, 1, 1, 1], [0, 0, 1, 1, 1], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0]]
4 [[0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 0]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[0, 7], [3, 1]]
1 [[2, 0], [1, 4], [0, 3]]
2 [[2, 3], [2, 4], [0, 1], [3, 0], [3, 6]]
3 [[2, 7], [0, 7], [1, 4], [3, 4]]
4 [[1, 0], [2, 7], [1, 4]]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[0, 0], [1, 1]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pformat(s, indent=2))

0 [ [1.7672360590835687, 0.0874078335792472, 5.958031159512462],
  [-1.8129471157318644, -0.20098302392207934, 6.259389886161877]]
1 [ [-0.6915477720104448, -0.11514207623275907, -1.610752743648705],
  [4.100965047354076, 0.10585749996289029, 4.760367683397707]]
2 [ [4.892831160769497, 0.5440098595651608, 6.654208976635324],
  [3.5375338406917747, -0.3387849627328432, -3.685282563702179]]
3 [ [2.4055728863419974, -0.14507968584268616, -9.96200335370242],
  [2.5600079474250084, -0.2777925582481404, 8.866146071020548],
  [0.8547219934227259, 0.0014757232043656199, 7.54861929733266]]
4 [ [-1.576890941004474, -0.8377880269550957, -8.937911179428209],
  [-2.8306155408971563, 0.19025255102158156, -4.380291381957391]]
5 [ [4.659638169539847, 0.33820533419604737, -7.6616187068819475],
  [-0.817384482622681, 0.5934779388029945, 0.3164270969014371],
  [3.887638954140983, 0.6633175108499745, 0.9313218704091568],
  [1.7597929754598507, 0.6845426651641102, 4.1239353575466335],
  [-4.087311573718196

Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [7]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]), expected_num_mutations=3, mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
mutated: [[0.0, 0.0], [-0.9915357441273862, 1.8025157689968454], [-1.8934201113271871, -0.4095411883290463]]


# Other pipeline operators

Besides the aformentioned `apply_mutation`, segmented representations have other pipeline operators, which are:

* `add_segment()`, possibly add a new segment
* `remove_segment()`, possibly remove a segment
* `copy_segment()`, possibly select and copy an existing segment


In [8]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               remove_segment(probability=1.0), 
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop))

pop: [Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None)]
new_pop: [Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None)]


In [9]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               copy_segment(probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [0, 0], [1, 1]], None, None),
         Individual([[1, 1], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [0, 0]], None, None)]


In [10]:
# lastly, demonstrate add_segment, which generates an entirely new segment
test_sequence = [12345]  # just an arbitrary sequence for testing

def gen_sequence():
    """ return an arbitrary static test_sequence """
    return test_sequence

pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))

new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               add_segment(seq_initializer=gen_sequence, probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [1, 1], [12345]], None, None),
         Individual([[12345], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [12345], [1, 1]], None, None),
         Individual([[0, 0], [12345], [1, 1]], None, None),
         Individual([[0, 0], [12345], [1, 1]], None, None)]
